## LibSVM Guide

如何使用 LibSVM? 官网上有一个很好的教程。下面跟着教程来操作一遍。也加深下机器学
习的流程，以及对 SVM 的理解。

### 测试1

- 数据集：太空粒子
- #features: 4
- #class: 2
- #training data: 3089
- #testing data: 4000

先查看下训练和测试的数据：

- 训练数据

```
❯ head datasets/svm-guide-train.1
1 1:2.617300e+01 2:5.886700e+01 3:-1.894697e-01 4:1.251225e+02
1 1:5.707397e+01 2:2.214040e+02 3:8.607959e-02 4:1.229114e+02
1 1:1.725900e+01 2:1.734360e+02 3:-1.298053e-01 4:1.250318e+02
1 1:2.177940e+01 2:1.249531e+02 3:1.538853e-01 4:1.527150e+02
1 1:9.133997e+01 2:2.935699e+02 3:1.423918e-01 4:1.605402e+02
1 1:5.537500e+01 2:1.792220e+02 3:1.654953e-01 4:1.112273e+02
1 1:2.956200e+01 2:1.913570e+02 3:9.901439e-02 4:1.034076e+02
1 1:1.451200e+02 2:2.088600e+02 3:-1.760859e-01 4:1.542257e+02
1 1:3.849699e+01 2:4.146600e+01 3:-1.886419e-01 4:1.239661e+02
1 1:2.927699e+01 2:1.072510e+02 3:1.149632e-01 4:1.077885e+02
```

- 测试数据

```
❯ head datasets/svm-guide-test.1
0 1:4.236298e+00 2:2.198210e+01 3:-3.503797e-01 4:9.752163e+01
0 1:4.236298e+00 2:2.198210e+01 3:-3.503797e-01 4:9.752163e+01
0 1:1.657013e+00 2:8.066301e+01 3:2.126006e-01 4:6.866834e+01
0 1:2.804001e+00 2:2.235300e+01 3:3.414961e-01 4:8.095125e+01
0 1:2.835701e+00 2:3.020160e+01 3:3.855923e-01 4:9.489861e+01
0 1:1.011299e+00 2:3.639751e+01 3:-1.509783e-01 4:6.572945e+01
0 1:2.585200e+01 2:4.267701e+01 3:1.589062e-01 4:6.920668e+01
0 1:1.535400e+00 2:3.526780e+01 3:2.476902e-01 4:6.275486e+01
0 1:9.810982e-01 2:4.357700e+01 3:1.706926e-01 4:1.115596e+02
0 1:5.011002e+00 2:3.365700e+01 3:2.070035e-01 4:8.448615e+01
```
#### 1. 未经过任何处理的测试

1. 生成模型

```
# 生成模型, 会输出训练信息，也会将模型保存为 svm-guide-train.1.model 文件中
$ ../svm-train ./datasets/svm-guide-train.1
...*..*
optimization finished, #iter = 5371
nu = 0.606150
obj = -1061.528918, rho = -0.495266
nSV = 3053, nBSV = 722
Total nSV = 3053

```

2. 查看模型


```
❯ head -15 svm-guide-train.1.model
svm_type c_svc
kernel_type rbf
gamma 0.25
nr_class 2
total_sv 3053
rho -0.49526608854956916
label 1 0
nr_sv 1996 1057
SV
0.27597478473293091 1:26.173 2:58.867 3:-0.1894697 4:125.1225
0.50424084723215956 1:57.07397 2:221.404 3:0.08607959 4:122.9114
0.50473434975028408 1:17.259 2:173.436 3:-0.1298053 4:125.0318
0.45086892077519203 1:21.7794 2:124.9531 3:0.1538853 4:152.715
0.50491587149854233 1:91.33997 2:293.5699 3:0.1423918 4:160.5402
0.5048770796985661 1:55.375 2:179.222 3:0.1654953 4:111.2273
```

其中：

- svm_type: c_svc 为分类器。
- kernel_type: rbf 核
- gamma: 0.25
- nr_class: number of class, 标签类别个数，2
- total_sv: sv 的个数 3053
- rho: -0.495
- label 1 0
- nr_sv: 1996 1057

3. 查看模型的准确率

```
❯ ../svm-predict datasets/svm-guide-test.1 svm-guide-train.1.model svm-guide-test.1.predict
Accuracy = 66.925% (2677/4000) (classification)

❯ head svm-guide-test.1.predict
0
0
1
1
1
0
1
0
1
1

```

可以看到模型的效果还是很差的。

那该怎么办那？我们先对数据做规一化处理。


### 测试1 优化

#### 1. 对数据做规一化，规一化区间为 [-1, 1]. 这也是默认的区间。

```
$ ../svm-scale -l -1 -u 1 -s svm-guide-train.1.range datasets/svm-guide-train.1 > svm-guide-train.1.norm

```

- svm-guide-trian.1.range 为规一化时的特征范围，我们需要保存下来。因为在测试时会用到。

```
$ cat svm-guide-train.1.range
x
-1 1
1 0 297.05000000000001
2 -4.5552060000000001 581.07309999999995
3 -0.75243850000000001 0.71706060000000005
4 8.1574740000000006 180

```

- svm-guide-train.1.norm 规一化后的训练数据

```
$ head svm-guide-train.1.norm
1 1:-0.823781 2:-0.783405 3:-0.233795 4:0.361305
1 1:-0.615728 2:-0.228319 3:0.14123 4:0.335571
1 1:-0.883797 2:-0.392136 3:-0.152591 4:0.360249
1 1:-0.853362 2:-0.557712 3:0.233514 4:0.682442
1 1:-0.38502 2:0.0181376 3:0.217871 4:0.773516
1 1:-0.627167 2:-0.372376 3:0.249315 4:0.199585
1 1:-0.800963 2:-0.330933 3:0.158834 4:0.108575
1 1:-0.0229254 2:-0.271158 3:-0.21558 4:0.700024
1 1:-0.740805 2:-0.842831 3:-0.232668 4:0.347846
1 1:-0.802882 2:-0.618167 3:0.180541 4:0.159562
```

下面就对规一化后的训练数据做下性能测试。

#### 2. 训练模型

```
❯ ../svm-train svm-guide-train.1.norm
*
optimization finished, #iter = 496
nu = 0.202599
obj = -507.307046, rho = 2.627039
nSV = 630, nBSV = 621
Total nSV = 630
```

可以看到经过规一化处理后，我们训练迭代测试减少了，支持向量也减少。

```
$ head -15 svm-guide-train.1.norm.model
svm_type c_svc
kernel_type rbf
gamma 0.25
nr_class 2
total_sv 630
rho 2.6270387466452201
label 1 0
nr_sv 316 314
SV
1 1:-0.823781 2:-0.783405 3:-0.233795 4:0.361305
1 1:-0.740805 2:-0.842831 3:-0.232668 4:0.347846
1 1:-0.838324 2:-0.724825 3:0.412097 4:0.0563095
1 1:-0.849309 2:-0.70294 3:-0.359031 4:-0.0476145
1 1:-1 2:-0.688314 3:0.595954 4:0.416735
1 1:-0.89062 2:-0.819471 3:0.773802 4:-0.78854

```

#### 3. 对测试数据进行规一化处理

```
$ ../svm-scale -r svm-guide-train.1.range ./datasets/svm-guide-test.1 > svm-guide-test.1.norm

$ head svm-guide-test.1.norm
0 1:-0.971478 2:-0.909372 3:-0.452795 4:0.0400703
0 1:-0.971478 2:-0.909372 3:-0.452795 4:0.0400703
0 1:-0.988844 2:-0.708968 3:0.313426 4:-0.29574
0 1:-0.981121 2:-0.908105 3:0.488854 4:-0.152785
0 1:-0.980908 2:-0.881301 3:0.548869 4:0.00954214
0 1:-0.993191 2:-0.860141 3:-0.181408 4:-0.329945
0 1:-0.825942 2:-0.838696 3:0.240347 4:-0.289475
0 1:-0.989662 2:-0.863999 3:0.361183 4:-0.364565
0 1:-0.993394 2:-0.835622 3:0.256389 4:0.203452
0 1:-0.966262 2:-0.8695 3:0.305808 4:-0.111644
```

#### 4. 测试

```
$ ../svm-predict svm-guide-test.1.norm svm-guide-train.1.norm.model svm-guide-test.1.norm.predict
Accuracy = 96.15% (3846/4000) (classification)
```

对数据进行规一化处理后，我们训练迭代测试减少了，支持向量也减少。准确率却大大提高了。


#### 5. 参数选择

那对于 SVM 我们松驰变量 C , 和 RBF 核函数的 gamma 取什么值好那？我们可以使用网格搜索的方法来寻找。

即执行如下命令：

```
$ ../tools/grid.py svm-guide-train.1.norm
gnuplot executable not found
[local] 5 -7 95.5973 (best c=32.0, g=0.0078125, rate=95.5973)
[local] -1 -7 85.2056 (best c=32.0, g=0.0078125, rate=95.5973)
[local] 5 -13 85.3027 (best c=32.0, g=0.5, rate=96.8598)
[local] -1 -13 64.7459 (best c=32.0, g=0.5, rate=96.8598)
.....
local] 13 -5 96.6008 (best c=8.0, g=2.0, rate=96.9246)
[local] 13 -15 95.4354 (best c=8.0, g=2.0, rate=96.9246)
[local] 13 3 94.9822 (best c=8.0, g=2.0, rate=96.9246)
[local] 13 -9 96.1476 (best c=8.0, g=2.0, rate=96.9246)
[local] 13 -3 96.7627 (best c=8.0, g=2.0, rate=96.9246)
8.0 2.0 96.9246

```

./grid.py 会使用 5-fold 交叉验证集来寻找到最优的参数 （c = 8.0, g = 2.0)

#### 6. 根据 grid.py 搜索的参数来重新训练

```
$ ../svm-train -c 8 -g 2 svm-guide-train.1.norm
*.*
optimization finished, #iter = 1113
nu = 0.088356
obj = -1950.406413, rho = 0.335733
nSV = 301, nBSV = 251
Total nSV = 301

~/code/github.com/libsvm/examples master*
❯ ../svm-predict svm-guide-test.1.norm svm-guide-train.1.norm.model svm-guide-test.1.norm.predict
Accuracy = 96.95% (3878/4000) (classification)

```

可以看到我们经过网络搜索和多重验证集之后，我们可以得到 96.95% 的正确率。

#### 7. 使用自动化的脚本

该脚本可以将数据规一化，参数空间搜索统一自动处理。
